In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
import os
import sys
import tensorflow as tf
import librosa
import librosa.display
import seaborn as sns
import matplotlib.pyplot as plt

/opt/homebrew/Caskroom/miniforge/base/envs/prismatic/lib/python3.10/site-packages/lazy_loader/__init__.py:185: RuntimeWarning: subpackages can technically be lazily loaded, but it causes the package to be eagerly loaded even if it is already lazily loaded.So, you probably shouldn't use subpackages with this lazy feature.
  warnings.warn(msg, RuntimeWarning)
/opt/homebrew/Caskroom/miniforge/base/envs/prismatic/lib/python3.10/site-packages/lazy_loader/__init__.py:185: RuntimeWarning: subpackages can technically be lazily loaded, but it causes the package to be eagerly loaded even if it is already lazily loaded.So, you probably shouldn't use subpackages with this lazy feature.
  warnings.warn(msg, RuntimeWarning)


In [3]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

# to play the audio files
from IPython.display import Audio

import keras
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [4]:
dataset_path = './data/features_age_emotion.csv'

data = pd.read_csv(dataset_path)

In [5]:
data['age'] = pd.cut(data['age'], bins=[20, 29, 39, 49, 59, 69, 79],
                           labels=['twenties', 'thirties', 'forties', 'fifties', 'sixties', 'seventies'],
                           right=False)

In [6]:
data = data.replace(np.nan, 0)

In [7]:
data['gender'] = data['gender'].replace("female",0)
data['gender'] = data['gender'].replace("male",1)

In [8]:
data.emotion.value_counts()

emotion
anger           5592
happiness       5368
anxiety/fear    5360
sadness         5332
disgust         5268
neutral         4664
Name: count, dtype: int64

In [9]:
# encoded_df = pd.get_dummies(data['emotion'])
# df_encoded = pd.concat([data, encoded_df], axis=1)
# df_encoded.head()
# data.head()

In [10]:
encoded_df = pd.get_dummies(data['age'])
df_encoded = pd.concat([data, encoded_df], axis=1)
df_encoded.head()

,0,1,2,3,4,5,6,7,8,9,...,2859,age,gender,emotion,twenties,thirties,forties,fifties,sixties,seventies
0,0.017090,0.024902,0.028809,0.045410,0.074707,0.100098,0.119141,0.110840,0.088867,0.067383,...,0.0,fifties,1,anger,False,False,False,True,False,False
1,0.182129,0.256836,0.350098,0.351074,0.281738,0.240234,0.170410,0.110840,0.089844,0.068359,...,0.0,fifties,1,anger,False,False,False,True,False,False
2,0.015625,0.029297,0.070801,0.104004,0.132324,0.149414,0.131836,0.109863,0.092773,0.076660,...,0.0,fifties,1,anger,False,False,False,True,False,False
3,0.076660,0.113770,0.167969,0.180176,0.173340,0.166992,0.136719,0.109863,0.091309,0.075684,...,0.0,fifties,1,anger,False,False,False,True,False,False
4,0.015137,0.031738,0.036621,0.041504,0.037109,0.026855,0.029297,0.029297,0.035645,0.067383,...,0.0,fifties,1,disgust,False,False,False,True,False,False


In [11]:
df_encoded = df_encoded.drop(['gender','emotion'], axis=1)
df_encoded.head()

,0,1,2,3,4,5,6,7,8,9,...,2857,2858,2859,age,twenties,thirties,forties,fifties,sixties,seventies
0,0.017090,0.024902,0.028809,0.045410,0.074707,0.100098,0.119141,0.110840,0.088867,0.067383,...,0.0,0.0,0.0,fifties,False,False,False,True,False,False
1,0.182129,0.256836,0.350098,0.351074,0.281738,0.240234,0.170410,0.110840,0.089844,0.068359,...,0.0,0.0,0.0,fifties,False,False,False,True,False,False
2,0.015625,0.029297,0.070801,0.104004,0.132324,0.149414,0.131836,0.109863,0.092773,0.076660,...,0.0,0.0,0.0,fifties,False,False,False,True,False,False
3,0.076660,0.113770,0.167969,0.180176,0.173340,0.166992,0.136719,0.109863,0.091309,0.075684,...,0.0,0.0,0.0,fifties,False,False,False,True,False,False
4,0.015137,0.031738,0.036621,0.041504,0.037109,0.026855,0.029297,0.029297,0.035645,0.067383,...,0.0,0.0,0.0,fifties,False,False,False,True,False,False


In [12]:
# encoded_df = pd.get_dummies(data['gender'])
# df_encoded = pd.concat([df_encoded, encoded_df], axis=1)
# df_encoded.head()

In [13]:
# df_encoded.rename(columns={df_encoded.columns[-2]: 'female', df_encoded.columns[-1]: 'male'}, inplace=True)
# df_encoded.head()

In [14]:
df_encoded.drop(['age'],axis=1,inplace=True)
df_encoded.head()

,0,1,2,3,4,5,6,7,8,9,...,2856,2857,2858,2859,twenties,thirties,forties,fifties,sixties,seventies
0,0.017090,0.024902,0.028809,0.045410,0.074707,0.100098,0.119141,0.110840,0.088867,0.067383,...,0.0,0.0,0.0,0.0,False,False,False,True,False,False
1,0.182129,0.256836,0.350098,0.351074,0.281738,0.240234,0.170410,0.110840,0.089844,0.068359,...,0.0,0.0,0.0,0.0,False,False,False,True,False,False
2,0.015625,0.029297,0.070801,0.104004,0.132324,0.149414,0.131836,0.109863,0.092773,0.076660,...,0.0,0.0,0.0,0.0,False,False,False,True,False,False
3,0.076660,0.113770,0.167969,0.180176,0.173340,0.166992,0.136719,0.109863,0.091309,0.075684,...,0.0,0.0,0.0,0.0,False,False,False,True,False,False
4,0.015137,0.031738,0.036621,0.041504,0.037109,0.026855,0.029297,0.029297,0.035645,0.067383,...,0.0,0.0,0.0,0.0,False,False,False,True,False,False


In [15]:
X = df_encoded.drop(['twenties', 'thirties', 'forties', 'fifties', 'sixties', 'seventies'], axis=1).values

In [16]:
y = df_encoded[['twenties', 'thirties', 'forties', 'fifties', 'sixties', 'seventies']].values


In [17]:
# y_emotion = df_encoded.iloc[:, -14:-8].values  # Emotion labels
y_age = df_encoded.iloc[:, -6:].values  # Age labels
# y_gender = df_encoded.iloc[:, -2:].values  # Gender labels

In [18]:
# y_gender.shape, y_emotion.shape, y_age.shape

In [19]:
x_train, x_val_test, y_age_train, y_age_val_test = train_test_split(X, y_age, test_size=0.3, random_state=30)
x_test, x_val, y_age_test, y_age_val = train_test_split(x_val_test, y_age_val_test, test_size=0.5, random_state=30)


In [20]:
scaler = StandardScaler()

In [21]:
x_train = scaler.fit_transform(x_train)
x_val=scaler.transform(x_val)
x_test = scaler.transform(x_test)

x_train.shape, x_test.shape, x_val.shape

((22108, 2860), (4738, 2860), (4738, 2860))

In [22]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv1D, BatchNormalization, MaxPooling1D, AveragePooling1D, Dropout, Flatten, Dense, Input

input_layer = Input(shape=(x_train.shape[1], 1))

# 1
hl = Conv1D(512, kernel_size=3, strides=1, padding='same', activation='relu')(input_layer)
hl = BatchNormalization()(hl)
hl = AveragePooling1D(pool_size=3, strides=2, padding='same')(hl)
hl = Dropout(0.3)(hl)

#2

hl = Conv1D(256, kernel_size=3, strides=1, padding='same', activation='relu')(hl)
hl = BatchNormalization()(hl)
hl = AveragePooling1D(pool_size=3, strides=2, padding='same')(hl)
hl = Dropout(0.3)(hl)

#3

hl = Conv1D(128, kernel_size=3, strides=1, padding='same', activation='relu')(hl)
hl = BatchNormalization()(hl)
hl = AveragePooling1D(pool_size=3, strides=2, padding='same')(hl)
hl = Dropout(0.3)(hl)

# Flatten layer
hl = Flatten()(hl)

# Dense layers for the shared layer
hl = Dense(64, activation='relu')(hl)
hl = BatchNormalization()(hl)
hl = Dropout(0.3)(hl)

# Output layers for each task
# emotion_output = Dense(y_emotion_train.shape[1], activation='softmax', name='emotion')(hl)
age_output = Dense(y_age_train.shape[1], activation='softmax', name='age')(hl)
# gender_output = Dense(y_gender_train.shape[1], activation='softmax', name='gender')(hl)

# Create the model with multiple outputs
model = Model(inputs=input_layer, outputs=[age_output])

2023-09-07 16:27:07.044894: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-09-07 16:27:07.045280: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



In [23]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 2860, 1)]         0         
                                                                 
 conv1d (Conv1D)             (None, 2860, 512)         2048      
                                                                 
 batch_normalization (BatchN  (None, 2860, 512)        2048      
 ormalization)                                                   
                                                                 
 average_pooling1d (AverageP  (None, 1430, 512)        0         
 ooling1D)                                                       
                                                                 
 dropout (Dropout)           (None, 1430, 512)         0         
                                                                 
 conv1d_1 (Conv1D)           (None, 1430, 256)         393472

In [24]:
# early_stop_emotion = EarlyStopping(monitor='val_emotion_accuracy', mode='auto', patience=10, restore_best_weights=True)
early_stop_age = EarlyStopping(monitor='val_accuracy', mode='auto', patience=10, restore_best_weights=True)
# early_stop_gender = EarlyStopping(monitor='val_accuracy', mode='auto', patience=10, restore_best_weights=True)


In [25]:
# lr_reduction_emotion = ReduceLROnPlateau(monitor='val_emotion_accuracy', patience=3, verbose=1, factor=0.5)
lr_reduction_age = ReduceLROnPlateau(monitor='val_accuracy', patience=3, verbose=1, factor=0.5)
# lr_reduction_gender = ReduceLROnPlateau(monitor='val_accuracy', patience=3, verbose=1, factor=0.5)


In [26]:
callbacks = [
    # lr_reduction_emotion,
    lr_reduction_age,
    # lr_reduction_gender,
    # early_stop_emotion,
    early_stop_age,
    # early_stop_gender
]

In [27]:
optimizer = keras.optimizers.Nadam()

In [28]:
model.compile(optimizer=optimizer,
              loss={
                  # 'emotion': 'categorical_crossentropy',
                    'age': 'categorical_crossentropy',
                    # 'gender': 'binary_crossentropy'
                    },
              metrics={
                  # 'emotion': 'accuracy',
                       'age': 'accuracy',
                      #  'gender': 'accuracy'
                       })

In [29]:
history = model.fit(x_train, {
                            # 'emotion': y_emotion_train,
                              'age': y_age_train,
                              # 'gender': y_gender_train
                              },

          validation_data=(x_val, {
                                # 'emotion': y_emotion_val,
                                   'age': y_age_val,
                                  #  'gender': y_gender_val
                                   }),

          epochs=200, batch_size=16, callbacks=callbacks)

Epoch 1/200


2023-09-07 16:27:08.070018: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-09-07 16:27:08.868900: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1382/1382 [==============================] - ETA: 0s - loss: 1.5877 - accuracy: 0.3232

2023-09-07 16:28:59.697543: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1382/1382 [==============================] - 117s 81ms/step - loss: 1.5877 - accuracy: 0.3232 - val_loss: 1.5685 - val_accuracy: 0.2879 - lr: 0.0010
Epoch 2/200
1382/1382 [==============================] - 129s 93ms/step - loss: 1.4532 - accuracy: 0.3578 - val_loss: 1.5053 - val_accuracy: 0.3356 - lr: 0.0010
Epoch 3/200
1382/1382 [==============================] - 119s 86ms/step - loss: 1.3986 - accuracy: 0.3857 - val_loss: 1.3542 - val_accuracy: 0.4097 - lr: 0.0010
Epoch 4/200
1382/1382 [==============================] - 108s 78ms/step - loss: 1.3361 - accuracy: 0.4213 - val_loss: 1.3044 - val_accuracy: 0.4365 - lr: 0.0010
Epoch 5/200
1382/1382 [==============================] - 106s 77ms/step - loss: 1.2611 - accuracy: 0.4624 - val_loss: 1.2853 - val_accuracy: 0.4320 - lr: 0.0010
Epoch 6/200
1382/1382 [==============================] - 106s 77ms/step - loss: 1.1882 - accuracy: 0.4994 - val_loss: 1.5072 - val_accuracy: 0.4496 - lr: 0.0010
Epoch 7/200
1382/1382 [=======================

In [30]:
model.save("age_model.keras")

In [32]:
losses = model.evaluate(x_test, {
                                # 'emotion': y_emotion_test,
                                 'age': y_age_test,
                                #  'gender': y_gender_test
                                 })
print('Emotion Loss:', losses[1])
print('Age Loss:', losses[2])
print('Gender Loss:', losses[3])
print('Emotion Accuracy:', losses[4])
print('Age Accuracy:', losses[5])
print('Gender Accuracy:', losses[6])

149/149 [==============================] - 5s 32ms/step - loss: 0.2027 - accuracy: 0.9466
Emotion Loss: 0.946601927280426


IndexError: list index out of range

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
y_emotion_pred = np.argmax(y_pred[0], axis=1)
y_age_pred = np.argmax(y_pred[1], axis=1)
y_gender_pred = (y_pred[2] > 0.5).astype(int)

In [ ]:
set(y_age_pred)

In [ ]:
y_gender_test.shape, y_age_test.shape, y_emotion_test.shape

In [ ]:
emotion_cm = confusion_matrix(np.argmax(y_emotion_test, axis=1), y_emotion_pred)
age_cm = confusion_matrix(np.argmax(y_age_test, axis=1), y_age_pred)
gender_cm = confusion_matrix(np.argmax(y_gender_test, axis=1), np.argmax(y_gender_pred, axis=1))

In [ ]:
labels_emotion = ['Anger', 'Anxiety/Fear', 'Disgust', 'Happiness', 'Neutral', 'Sadness']
labels_age = ['Twenties', 'Thirties', 'Forties', 'Fifties', 'Sixties', 'Seventies']
labels_gender = ['Male', 'Female']

fig, axes = plt.subplots(1, 3, figsize=(20, 6))
sns.heatmap(emotion_cm, annot=True, cmap='Blues', fmt='d', ax=axes[0])
axes[0].set_xticks(np.arange(len(labels_emotion)) + 0.5)
axes[0].set_yticks(np.arange(len(labels_emotion)) + 0.5)
axes[0].set_xticklabels(labels_emotion, rotation=45)
axes[0].set_yticklabels(labels_emotion)
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('True')
axes[0].set_title('Emotion Confusion Matrix')

sns.heatmap(age_cm, annot=True, cmap='Blues', fmt='d', ax=axes[1])
axes[1].set_xticks(np.arange(len(labels_age)) + 0.5)
axes[1].set_yticks(np.arange(len(labels_age)) + 0.5)
axes[1].set_xticklabels(labels_age, rotation=45)
axes[1].set_yticklabels(labels_age)
axes[1].set_xlabel('Predicted')
axes[1].set_ylabel('True')
axes[1].set_title('Age Confusion Matrix')

sns.heatmap(gender_cm, annot=True, cmap='Blues', fmt='d', ax=axes[2])
axes[2].set_xticks(np.arange(len(labels_gender)) + 0.5)
axes[2].set_yticks(np.arange(len(labels_gender)) + 0.5)
axes[2].set_xticklabels(labels_gender)
axes[2].set_yticklabels(labels_gender)
axes[2].set_xlabel('Predicted')
axes[2].set_ylabel('True')
axes[2].set_title('Gender Confusion Matrix')


plt.tight_layout()
plt.show()

In [ ]:
print(classification_report(np.argmax(y_emotion_test, axis=1), y_emotion_pred))
print(classification_report(np.argmax(y_age_test, axis=1), y_age_pred))
print(classification_report(y_gender_test, y_gender_pred, target_names=['male','female']))